In [1]:
import ThreeD_Net_Tools_0203 as D3

import sys
import math
import numpy as np #v 1.23.3
import networkx as nx
import matplotlib.cm as cm
import matplotlib
import argparse
import matplotlib.pyplot as plt
import itertools
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from networkx.algorithms import community
import community
from community import community_louvain #conda install python-louvain
from PIL import Image
import os
import scipy
from numpy import roots
import random

%matplotlib notebook

In [2]:
USR='LUKE'

In [3]:
#os.getcwd()
list_paths_skl=[]
list_paths_vol=[]

if USR=='LUKE':
    ALL_ANALYSIS='ALL_GAUDI.csv'#finalfilename
    directory="/media/ll16598/One Touch/CT_ANALYSIS"
    dir_G="/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/BASE"
    dir_G_inverted='/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/INVERTED'
    dir_G_widths='/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS'
    dir_G_widths_inverted='/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS_INVERTED'
    dir_G_short="/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/SHORT"
    dir_G_spatial="/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/SPATIAL"
    dir_G_short_widths="/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/SHORT_WIDTH"
    dir_G_spatial_widths="/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/SPATIAL_WIDTH"
    
    os.chdir("/media/ll16598/One Touch/CT_ANALYSIS")
    VOXEL_SIZES = pd.read_csv('/home/ll16598/Documents/ARCHITECTURAL_IMMUNITY/ANALYSIS/VOXEL_SIZES.csv')

elif USR=='SUPER':
    directory="/media/cf19810/One Touch/CT_ANALYSIS"
    dir_G="/media/cf19810/One Touch/CT_ANALYSIS/GRAPHS/BASE"
    dir_G_inverted='/media/cf19810/One Touch/CT_ANALYSIS/GRAPHS/INVERTED'
    dir_G_widths='/media/cf19810/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS'
    dir_G_widths_inverted='/media/cf19810/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS_INVERTED'
    dir_G_short="/media/cf19810/One Touch/CT_ANALYSIS/GRAPHS/SHORT"
    dir_G_spatial="/media/cf19810/One Touch/CT_ANALYSIS/GRAPHS/SPATIAL"
    dir_G_short_widths="/media/cf19810/One Touch/CT_ANALYSIS/GRAPHS/SHORT_WIDTH"
    dir_G_spatial_widths="/media/cf19810/One Touch/CT_ANALYSIS/GRAPHS/SPATIAL_WIDTH"
    
    os.chdir("/media/cf19810/One Touch/CT_ANALYSIS")
    VOXEL_SIZES = pd.read_csv('/home/cf19810/Documents/LUKE/VOXEL_SIZES.csv')

In [4]:
TREATS = pd.read_csv('/home/ll16598/Documents/ARCHITECTURAL_IMMUNITY/ANALYSIS/COLONY_INFO.csv')
analysis_df=TREATS

In [5]:
name_list=[]
for i in TREATS['name']:
    name_list.append(i)

In [6]:
# dir_G='/home/ll16598/Documents/ARCHITECTURAL_IMMUNITY/GRAPHS/WIDTHS'
# dir_G_short='/home/ll16598/Documents/ARCHITECTURAL_IMMUNITY/GRAPHS/SHORT_WIDTH'
# dir_G_spatial='/home/ll16598/Documents/ARCHITECTURAL_IMMUNITY/GRAPHS/SPATIAL_WIDTH'

ENSURE YOU ARE RUNNING THE CORRECT GRAPHS. WIDTHS OR INVERTED.

In [7]:
WIDTH=True

In [8]:
if WIDTH==True:
    files = [ f.path for f in os.scandir(dir_G_widths)]
    files_inverted = [ f.path for f in os.scandir(dir_G_widths_inverted)]
    print('WIDTH TAKEN INTO ACCOUNT')
elif WIDTH==False:
    print('WIDTH NOT TAKEN INTO ACCOUNT')
    files = [ f.path for f in os.scandir(dir_G)]
    files_inverted = [ f.path for f in os.scandir(dir_G_inverted)]
#files_short = [ f.path for f in os.scandir(dir_G_short)]
#files_spatial = [ f.path for f in os.scandir(dir_G_spatial)]

WIDTH TAKEN INTO ACCOUNT


Reading in the graphs. Graphs with _short_ are junction removed with shortest path length based on edge number between nodes. _spatial_ are based on spatial distance between nodes, using Dijkstra.

In [9]:
files

['/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C1SK_TUE_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C1SK_WED_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C1SK_THU_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C1SK_FRI_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C1SK_MON_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C2PM_TUE_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C2PM_WED_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C2PM_THU_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C2PM_FRI_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C2PM_MON_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R2C1PK_TUE_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R2C1PK_WED_graph_widths.xml',
 '/m

In [10]:
name_list

['R1C1SK_TUE',
 'R1C1SK_WED',
 'R1C1SK_THU',
 'R1C1SK_FRI',
 'R1C1SK_MON',
 'R1C2PM_TUE',
 'R1C2PM_WED',
 'R1C2PM_THU',
 'R1C2PM_FRI',
 'R1C2PM_MON',
 'R2C1PK_TUE',
 'R2C1PK_WED',
 'R2C1PK_THU',
 'R2C1PK_FRI',
 'R2C1PK_MON',
 'R2C2SM_TUE',
 'R2C2SM_WED',
 'R2C2SM_THU',
 'R2C2SM_FRI',
 'R2C2SM_MON',
 'R3C1SK_TUE',
 'R3C1SK_WED',
 'R3C1SK_THU',
 'R3C1SK_FRI',
 'R3C1SK_MON',
 'R3C2PM_TUE',
 'R3C2PM_WED',
 'R3C2PM_THU',
 'R3C2PM_FRI',
 'R3C2PM_MON',
 'R4C1PK_TUE',
 'R4C1PK_WED',
 'R4C1PK_THU',
 'R4C1PK_FRI',
 'R4C1PK_MON',
 'R4C2SM_TUE',
 'R4C2SM_WED',
 'R4C2SM_THU',
 'R4C2SM_FRI',
 'R4C2SM_MON',
 'R5C1SK_TUE',
 'R5C1SK_WED',
 'R5C1SK_THU',
 'R5C1SK_FRI',
 'R5C1SK_MON',
 'R5C2PM_TUE',
 'R5C2PM_WED',
 'R5C2PM_THU',
 'R5C2PM_FRI',
 'R5C2PM_MON',
 'R6C1PK_TUE',
 'R6C1PK_WED',
 'R6C1PK_THU',
 'R6C1PK_FRI',
 'R6C1PK_MON',
 'R6C2SM_TUE',
 'R6C2SM_WED',
 'R6C2SM_THU',
 'R6C2SM_FRI',
 'R6C2SM_MON',
 'R7C1SK_TUE',
 'R7C1SK_WED',
 'R7C1SK_THU',
 'R7C1SK_FRI',
 'R7C1SK_MON',
 'R7C2PM_TUE',
 'R7C2PM_W

In [11]:
files

['/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C1SK_TUE_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C1SK_WED_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C1SK_THU_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C1SK_FRI_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C1SK_MON_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C2PM_TUE_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C2PM_WED_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C2PM_THU_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C2PM_FRI_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R1C2PM_MON_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R2C1PK_TUE_graph_widths.xml',
 '/media/ll16598/One Touch/CT_ANALYSIS/GRAPHS/WIDTHS/R2C1PK_WED_graph_widths.xml',
 '/m

In [12]:
G_list=[]
all_names=[]
l=0
while l<len(files):
    for file in range(0,len(files)):
        day=os.path.basename(files[file])
        filename = "_".join(day.split("_")[:2])  # Split by underscore, take first two parts, join with underscore
        print(file)
        
        if filename == name_list[l]:
            G=nx.read_graphml(files[file])
            Gs = sorted(nx.connected_components(G), key=len, reverse=True)
            Gmax = G.subgraph(Gs[0])
            G_list.append(Gmax)
            print(filename)
            all_names.append(filename)
            l+=1
        else:
            l+=0.5
           # print(filename, name_list[l])
#             break

0
R1C1SK_TUE
1
R1C1SK_WED
2
R1C1SK_THU
3
R1C1SK_FRI
4
R1C1SK_MON
5
R1C2PM_TUE
6
R1C2PM_WED
7
R1C2PM_THU
8
R1C2PM_FRI
9
R1C2PM_MON
10
R2C1PK_TUE
11
R2C1PK_WED
12
R2C1PK_THU
13
R2C1PK_FRI
14
R2C1PK_MON
15
R2C2SM_TUE
16
R2C2SM_WED
17
R2C2SM_THU
18
R2C2SM_FRI
19
R2C2SM_MON
20
R3C1SK_TUE
21
R3C1SK_WED
22
R3C1SK_THU
23
R3C1SK_FRI
24
R3C1SK_MON
25
R3C2PM_TUE
26
R3C2PM_WED
27
R3C2PM_THU
28
R3C2PM_FRI
29
R3C2PM_MON
30
R4C1PK_TUE
31
R4C1PK_WED
32
R4C1PK_THU
33
R4C1PK_FRI
34
R4C1PK_MON
35
R4C2SM_TUE
36
R4C2SM_WED
37
R4C2SM_THU
38
R4C2SM_FRI
39
R4C2SM_MON
40
R5C1SK_TUE
41
R5C1SK_WED
42
R5C1SK_THU
43
R5C1SK_FRI
44
R5C1SK_MON
45
R5C2PM_TUE
46
R5C2PM_WED
47
R5C2PM_THU
48
R5C2PM_FRI
49
R5C2PM_MON
50
R6C1PK_TUE
51
R6C1PK_WED
52
R6C1PK_THU
53
R6C1PK_FRI
54
R6C1PK_MON
55
R6C2SM_TUE
56
R6C2SM_WED
57
R6C2SM_THU
58
R6C2SM_FRI
59
R6C2SM_MON
60
R7C1SK_TUE
61
R7C1SK_WED
62
R7C1SK_THU
63
R7C1SK_FRI
64
R7C1SK_MON
65
R7C2PM_TUE
66
R7C2PM_WED
67
R7C2PM_THU
68
R7C2PM_FRI
69
R7C2PM_MON
70
R8C1PL_TUE
71
R8C1PL_WED
72

In [13]:
G_list_inverted=[]
all_names_inverted=[]
l=0
while l<len(files_inverted):
    for file in range(0,len(files_inverted)):
        day=os.path.basename(files_inverted[file])
        filename = "_".join(day.split("_")[:2])  # Split by underscore, take first two parts, join with underscore
        print(file)
        
        if filename == name_list[l]:
            G=nx.read_graphml(files_inverted[file])
            Gs = sorted(nx.connected_components(G), key=len, reverse=True)
            Gmax = G.subgraph(Gs[0])
            G_list_inverted.append(Gmax)
            print(filename)
            all_names_inverted.append(filename)
            l+=1
        else:
            l+=0.5
           # print(filename, name_list[l])
#             break

0
R1C1SK_TUE
1
R1C1SK_WED
2
R1C1SK_THU
3
R1C1SK_FRI
4
R1C1SK_MON
5
R1C2PM_TUE
6
R1C2PM_WED
7
R1C2PM_THU
8
R1C2PM_FRI
9
R1C2PM_MON
10
R2C1PK_TUE
11
R2C1PK_WED
12
R2C1PK_THU
13
R2C1PK_FRI
14
R2C1PK_MON
15
R2C2SM_TUE
16
R2C2SM_WED
17
R2C2SM_THU
18
R2C2SM_FRI
19
R2C2SM_MON
20
R3C1SK_TUE
21
R3C1SK_WED
22
R3C1SK_THU
23
R3C1SK_FRI
24
R3C1SK_MON
25
R3C2PM_TUE
26
R3C2PM_WED
27
R3C2PM_THU
28
R3C2PM_FRI
29
R3C2PM_MON
30
R4C1PK_TUE
31
R4C1PK_WED
32
R4C1PK_THU
33
R4C1PK_FRI
34
R4C1PK_MON
35
R4C2SM_TUE
36
R4C2SM_WED
37
R4C2SM_THU
38
R4C2SM_FRI
39
R4C2SM_MON
40
R5C1SK_TUE
41
R5C1SK_WED
42
R5C1SK_THU
43
R5C1SK_FRI
44
R5C1SK_MON
45
R5C2PM_TUE
46
R5C2PM_WED
47
R5C2PM_THU
48
R5C2PM_FRI
49
R5C2PM_MON
50
R6C1PK_TUE
51
R6C1PK_WED
52
R6C1PK_THU
53
R6C1PK_FRI
54
R6C1PK_MON
55
R6C2SM_TUE
56
R6C2SM_WED
57
R6C2SM_THU
58
R6C2SM_FRI
59
R6C2SM_MON
60
R7C1SK_TUE
61
R7C1SK_WED
62
R7C1SK_THU
63
R7C1SK_FRI
64
R7C1SK_MON
65
R7C2PM_TUE
66
R7C2PM_WED
67
R7C2PM_THU
68
R7C2PM_FRI
69
R7C2PM_MON
70
R8C1PL_TUE
71
R8C1PL_WED
72

In [14]:
# G_list_short=[]
# all_names=[]
# completed=False
# l=0
# while l<len(files_short):
#     for file in range(0,len(files_short)):
#         day=os.path.basename(files_short[file])
#         filename = "_".join(day.split("_")[:2])  # Split by underscore, take first two parts, join with underscore
#         if filename == name_list[l]:
#             G=nx.read_graphml(files_short[file])
#             Gs = sorted(nx.connected_components(G), key=len, reverse=True)
#             attributes = nx.get_node_attributes(G_list[l], 'TYPE')
#             nx.set_node_attributes(G, attributes, name='TYPE')
#             Gmax = G.subgraph(Gs[0])
#             G_list_short.append(Gmax)
#             print(filename)
#             all_names.append(filename)
#             l+=1
#         else:
#             continue

In [15]:
# G_list_spatial=[]
# all_names=[]
# completed=False
# l=0
# while l<len(files_short):
#     for file in range(0,len(files_short)):
#         day=os.path.basename(files_short[file])
#         filename = "_".join(day.split("_")[:2])  # Split by underscore, take first two parts, join with underscore
#         if filename == name_list[l]:
#             G=nx.read_graphml(files_short[file])
#             Gs = sorted(nx.connected_components(G), key=len, reverse=True)
#             attributes = nx.get_node_attributes(G_list[l], 'TYPE')
#             nx.set_node_attributes(G, attributes, name='TYPE')
#             Gmax = G.subgraph(Gs[0])
#             G_list_short.append(Gmax)
#             print(filename)
#             all_names.append(filename)
#             l+=1
#         else:
#             continue

In [16]:
def invert_edge_weights (G_list):
    G_list_inverted=[]
    for g in range(len(G_list)):
        G1=G_list[g]
        G=G1.copy(G1)
        for u, v, data in G.edges(data=True):
            data['weight'] = 1/data['weight']
        G_list_inverted.append(G)
    return G_list_inverted

#this will invert the og graph, so is not useable

In [17]:
#G_list_inverted =invert_edge_weights(G_list)
#G_list_inverted_short =invert_edge_weights(G_list_short)
#G_list_inverted_spatial =invert_edge_weights(G_list_spatial)

Now to create a simulation to test if random, naive, ants make it to chambers from entrances faster or slower in s/p.

In [18]:
its=1000

In [19]:
all_sum_weight_list=[]
all_edge_number_list=[]

for g in range(0, len(G_list)):
    G=G_list[g]
    print('GRAPH=', g)
    ants=0
    attributes = nx.get_node_attributes(G, 'TYPE')
    nest_nodes = [node for node, type_value in attributes.items() if type_value == 'NEST EN']
    chamber_nodes = [node for node, type_value in attributes.items() if type_value == 'CHAM']
    end_nodes = [node for node, type_value in attributes.items() if type_value == 'END']
    junc_nodes = [node for node, type_value in attributes.items() if type_value == 'JUNC']
    #sum_weights are the time taken to reach a chamber. Edge number is the number of edges passed
    sum_weight_list=[]
    edge_number_list=[]
    if len(chamber_nodes)==0:
        print('NO_CHAMBERS IN THIS G')
        sum_weights=np.nan
        path_length=np.nan
        sum_weight_list.append(sum_weights)
        edge_number_list.append(path_length)
        all_sum_weight_list.append(sum_weight_list)
        all_edge_number_list.append(edge_number_list)
        continue
        
    while ants < its:
        weights=[]
        #randomly choose a nest enrance, get its neighbours
        entrance=random.choice(nest_nodes)
        node1=entrance
        print('ENTRANCE=', entrance)
        chamber_reached=False
        while chamber_reached==False:

            neighbours=list(G.neighbors(node1))
            #it now randomly chooses a next node to move to
            node2=random.choice(neighbours)
            edge_data = G.get_edge_data(node1, node2)
            weight = edge_data['weight']
            weights.append(weight)
            if node2 in chamber_nodes:
                chamber_reached=True
                print('CHAMBER=', node2)
                ants+=1
                break
            neighbours=list(G.neighbors(node2))
            #print(node2, node1, neighbours)
            #if the ant reaches a nest entrance or an end point it turns around
            #tiny assumption of goal directedness
            if node2 not in end_nodes and node2 not in nest_nodes:
                neighbours=list(set(neighbours)-set(node2))
            node1=random.choice(neighbours)
            edge_data = G.get_edge_data(node1, node2)
            weight = edge_data['weight']
            weights.append(weight)
            if node1 in chamber_nodes:
                chamber_reached=True
                print('CHAMBER=', node1)
                ants+=1
                break


        sum_weights=np.sum(weights)
        path_length=len(weights)
        sum_weight_list.append(sum_weights)
        edge_number_list.append(path_length)
    
    all_sum_weight_list.append(sum_weight_list)
    all_edge_number_list.append(edge_number_list)



GRAPH= 0
ENTRANCE= 9
CHAMBER= 7
ENTRANCE= 12
CHAMBER= 7
ENTRANCE= 9
CHAMBER= 7
ENTRANCE= 12
CHAMBER= 7
ENTRANCE= 4
CHAMBER= 7
ENTRANCE= 12
CHAMBER= 7
ENTRANCE= 4
CHAMBER= 7
ENTRANCE= 8
CHAMBER= 7
ENTRANCE= 12
CHAMBER= 7
ENTRANCE= 8
CHAMBER= 7
ENTRANCE= 8
CHAMBER= 7
ENTRANCE= 12
CHAMBER= 7
ENTRANCE= 12
CHAMBER= 7
ENTRANCE= 12
CHAMBER= 7
ENTRANCE= 9
CHAMBER= 7
ENTRANCE= 12
CHAMBER= 7
ENTRANCE= 8
CHAMBER= 7
ENTRANCE= 8
CHAMBER= 7
ENTRANCE= 4
CHAMBER= 7
ENTRANCE= 8
CHAMBER= 7
ENTRANCE= 12
CHAMBER= 7
ENTRANCE= 12
CHAMBER= 7
ENTRANCE= 12
CHAMBER= 7
ENTRANCE= 8
CHAMBER= 7
ENTRANCE= 12
CHAMBER= 7
ENTRANCE= 4
CHAMBER= 7
ENTRANCE= 12
CHAMBER= 7
ENTRANCE= 4
CHAMBER= 7
ENTRANCE= 9
CHAMBER= 7
ENTRANCE= 12
CHAMBER= 7
ENTRANCE= 9
CHAMBER= 7
ENTRANCE= 4
CHAMBER= 7
ENTRANCE= 8
CHAMBER= 7
ENTRANCE= 9
CHAMBER= 7
ENTRANCE= 12
CHAMBER= 7
ENTRANCE= 8
CHAMBER= 7
ENTRANCE= 8
CHAMBER= 7
ENTRANCE= 12
CHAMBER= 7
ENTRANCE= 4
CHAMBER= 7
ENTRANCE= 12
CHAMBER= 7
ENTRANCE= 12
CHAMBER= 7
ENTRANCE= 8
CHAMBER= 7
ENTRANC

CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 21
CHAMBER= 7
ENTRANCE= 16
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 16
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 7
ENTRANCE= 21
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 44
CHAMBER= 7
ENTRANCE= 44
CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 21
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 21
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 7
ENTRANCE= 44
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 16
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 41
CHAMBER= 7
ENTRANCE= 21
CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRA

CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 44
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 7
ENTRANCE= 21
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 7
ENTRANCE= 21
CHAMBER= 7
ENTRANCE= 41
CHAMBER= 7
ENTRANCE= 16
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 21
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 16
CHAMBER= 7
ENTRANCE= 16
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 21
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 7
ENTRANCE= 16
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 44
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 7
ENTRANCE= 16
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 44
CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 41
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 7
ENTRANCE= 21
CHAMBER= 7
ENTRANCE= 16
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRA

CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 16
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 7
ENTRANCE= 16
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 41
CHAMBER= 7
ENTRANCE= 21
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 44
CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 41
CHAMBER= 7
ENTRANCE= 21
CHAMBER= 7
ENTRANCE= 16
CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 16
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 41
CHAMBER= 7
ENTRANCE= 41
CHAMBER= 7
ENTRANCE= 44
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 7
ENTRANCE= 16
CHAMBER= 7
ENTRANCE= 21
CHAMBER= 7
ENTRANCE= 16
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 16
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 41
CHAMBER= 7
ENTRANCE= 16
CHAMBER= 7
ENTRANCE= 44
CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 20
CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRA

CHAMBER= 7
ENTRANCE= 21
CHAMBER= 47
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 47
ENTRANCE= 21
CHAMBER= 47
ENTRANCE= 21
CHAMBER= 47
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 44
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 47
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 41
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 47
ENTRANCE= 55
CHAMBER= 47
ENTRANCE= 48
CHAMBER= 47
ENTRANCE= 41
CHAMBER= 7
ENTRANCE= 44
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 44
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 47
ENTRANCE= 55
CHAMBER= 7
ENTRANCE= 41
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 47
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 41
CHAMBER= 47
ENTRANCE= 37
CHAMBER= 7
ENTRANCE= 44
CHAMBER= 7
ENTRANCE= 48
CHAMBER= 47
ENTRANCE= 21
CHAMBER= 47
ENTRANCE= 21
CHAMBER= 47
ENTRANCE= 44
CHAMBER= 7
ENTRANCE= 41
CHAMBER= 47
ENTRANCE= 21
CHAMBER= 47
ENTRANCE= 48
CHAMBER= 47
ENTRANCE= 55
CHAMBER= 47
ENTRANCE= 44
CHAMBER= 7
ENTRANCE= 44
CHAMBER= 7
ENTRANCE= 55
CHAMBER= 47
ENTRANCE= 

...

CHAMBER= 26
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 35
ENTRANCE= 18
CHAMBER= 35
ENTRANCE= 16
CHAMBER= 33
ENTRANCE= 50
CHAMBER= 35
ENTRANCE= 16
CHAMBER= 35
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 33
ENTRANCE= 50
CHAMBER= 35
ENTRANCE= 49
CHAMBER= 35
ENTRANCE= 50
CHAMBER= 35
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 16
CHAMBER= 26
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 26
ENTRANCE= 24
CHAMBER= 33
ENTRANCE= 49
CHAMBER= 26
ENTRANCE= 49
CHAMBER= 33
ENTRANCE= 49
CHAMBER= 35
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 49
CHAMBER= 33
ENTRANCE= 49
CHAMBER= 35
ENTRANCE= 24
CHAMBER= 33
ENTRANCE= 50
CHAMBER= 33
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 24
CHAMBER= 35
ENTRANCE= 50
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 35
ENTRANCE= 16
CHAMBER= 33
ENTRANCE= 16
CHAMBER= 35
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 49
CHAMBER= 35
ENTRANCE= 16
CHAMBER= 33
ENTRANCE= 50


CHAMBER= 33
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 24
CHAMBER= 33
ENTRANCE= 49
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 33
ENTRANCE= 16
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 33
ENTRANCE= 50
CHAMBER= 35
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 24
CHAMBER= 33
ENTRANCE= 16
CHAMBER= 26
ENTRANCE= 16
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 35
ENTRANCE= 49
CHAMBER= 26
ENTRANCE= 49
CHAMBER= 33
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 33
ENTRANCE= 24
CHAMBER= 35
ENTRANCE= 50
CHAMBER= 33
ENTRANCE= 49
CHAMBER= 35
ENTRANCE= 16
CHAMBER= 33
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 16
CHAMBER= 33
ENTRANCE= 16
CHAMBER= 26
ENTRANCE= 49
CHAMBER= 33
ENTRANCE= 50
CHAMBER= 35
ENTRANCE= 49
CHAMBER= 26
ENTRANCE= 16
CHAMBER= 26
ENTRANCE= 16
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 33
ENTRANCE= 49
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 33
ENTRANCE= 49
CHAMBER= 33
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 49
CHAMBER= 35
ENTRANCE= 24
CHAMBER= 33
ENTRANCE= 50


CHAMBER= 65
ENTRANCE= 16
CHAMBER= 65
ENTRANCE= 49
CHAMBER= 35
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 24
CHAMBER= 65
ENTRANCE= 50
CHAMBER= 35
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 49
CHAMBER= 65
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 16
CHAMBER= 26
ENTRANCE= 49
CHAMBER= 65
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 16
CHAMBER= 65
ENTRANCE= 50
CHAMBER= 65
ENTRANCE= 49
CHAMBER= 65
ENTRANCE= 18
CHAMBER= 35
ENTRANCE= 16
CHAMBER= 26
ENTRANCE= 16
CHAMBER= 65
ENTRANCE= 49
CHAMBER= 65
ENTRANCE= 50
CHAMBER= 35
ENTRANCE= 16
CHAMBER= 35
ENTRANCE= 16
CHAMBER= 65
ENTRANCE= 16
CHAMBER= 35
ENTRANCE= 49
CHAMBER= 35
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 49
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 24
CHAMBER= 35
ENTRANCE= 50
CHAMBER= 35
ENTRANCE= 16
CHAMBER= 35
ENTRANCE= 16
CHAMBER= 26
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 16
CHAMBER= 33
ENTRANCE= 50
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 35
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 18


CHAMBER= 33
ENTRANCE= 50
CHAMBER= 65
ENTRANCE= 50
CHAMBER= 65
ENTRANCE= 18
CHAMBER= 65
ENTRANCE= 18
CHAMBER= 65
ENTRANCE= 50
CHAMBER= 35
ENTRANCE= 16
CHAMBER= 65
ENTRANCE= 24
CHAMBER= 35
ENTRANCE= 49
CHAMBER= 65
ENTRANCE= 50
CHAMBER= 35
ENTRANCE= 50
CHAMBER= 65
ENTRANCE= 18
CHAMBER= 65
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 26
ENTRANCE= 49
CHAMBER= 65
ENTRANCE= 18
CHAMBER= 35
ENTRANCE= 24
CHAMBER= 35
ENTRANCE= 49
CHAMBER= 65
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 35
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 65
ENTRANCE= 49
CHAMBER= 65
ENTRANCE= 49
CHAMBER= 35
ENTRANCE= 49
CHAMBER= 65
ENTRANCE= 49
CHAMBER= 65
ENTRANCE= 49
CHAMBER= 26
ENTRANCE= 49
CHAMBER= 26
ENTRANCE= 16
CHAMBER= 65
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 16
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 35
ENTRANCE= 16
CHAMBER= 26
ENTRANCE= 49
CHAMBER= 65
GRAPH= 99
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 37
CHAMBER= 2
ENTRANCE= 24
CHAMBER= 2
ENTRANCE= 50
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 2
ENTRAN

CHAMBER= 35
ENTRANCE= 50
CHAMBER= 2
ENTRANCE= 49
CHAMBER= 2
ENTRANCE= 37
CHAMBER= 2
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 26
ENTRANCE= 37
CHAMBER= 2
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 2
ENTRANCE= 49
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 37
CHAMBER= 2
ENTRANCE= 37
CHAMBER= 35
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 2
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 37
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 37
CHAMBER= 26
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 2
ENTRANCE= 49
CHAMBER= 26
ENTRANCE= 24
CHAMBER= 2
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 50
CHAMBER= 35
ENTRANCE= 50
CHAMBER= 2
ENTRANCE= 37
CHAMBER= 26
ENTRANCE= 24
CHAMBER= 2
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 24
CHAMBER= 26
ENTRANCE= 49
CHAMBER= 2
ENTRANCE= 49
CHAMBER= 26
ENTRANCE= 37
CHAMBER= 2
ENTRANCE= 18
CHAMBER= 26
ENTRANCE= 49
CHAMBER= 35
ENTRANCE= 18
CHAMBER= 35
E

In [43]:
#now save as df. Each path occupies its own cell so that cell character limit is not exceeded
simulation_df=pd.DataFrame(columns=['name', 'colony', 'order', 'week', 'day', 'subset', 'treatment'])
DFs=[]
for index, row in TREATS.iterrows():
    # Get corresponding values from list of lists
    sum_weight = all_sum_weight_list[index]
    edge_num = all_edge_number_list[index]
    # Iterate over values in list of lists and append new rows to new dataframe
    for i in range(len(sum_weight)):
        new_df = pd.DataFrame({
            'name': [TREATS['name'][index]],
            'colony': [TREATS['colony'][index]],
            'order': [TREATS['order'][index]],
            'week': [TREATS['week'][index]],
            'day': [TREATS['day'][index]],
            'treatment': [TREATS['treatment'][index]],
            'subset': [TREATS['subset'][index]],
            'sum_weight': sum_weight[i],
            'edge_num': edge_num[i] })
    
        DFs.append(new_df)
result= pd.concat(DFs)                      
simulation_df = result.reset_index(drop=True)

In [44]:
if WIDTH==True:
    simulation_df.to_csv(directory+"/"+'SIMULATION_ANALYSIS_WIDTH.csv',index=False)#S+PARAMETERS
elif WIDTH==False:
    simulation_df.to_csv(directory+"/"+'SIMULATION_ANALYSIS_BASE.csv',index=False)#S+PARAMETERS

This is the flow graph anlysis where entrances, chambers and endpoints can be treated as sources/sinks.

In [31]:
#FLOW ANALYSIS
print('BEGINNING FLOW GRAPH ANALYSIS')
ne_ch_flow_value_list=[]
ne_ch_edges_traversed_list=[]
ne_ch_chambers_traversed_list=[]
ne_ch_flow_weight_list=[]

for g in range(0, len(G_list_inverted)):
    G=G_list_inverted[g]
    Gs = sorted(nx.connected_components(G), key=len, reverse=True)
    Gmax = G.subgraph(Gs[0])
    G=Gmax
    DG=G.to_directed()

    ne_ch_flow_value=[]
    ne_ch_edges_traversed=[]
    ne_ch_chambers_traversed=[]
    ne_ch_flow_weight=[]

    attributes = nx.get_node_attributes(DG, 'TYPE')
    #     try:
    nest_nodes = [node for node, type_value in attributes.items() if type_value == 'NEST EN']
    chamber_nodes = [node for node, type_value in attributes.items() if type_value == 'CHAM']
    flow_value=[]
    edges_traversed=[]
    chambers_traversed=[]
    for nest in nest_nodes:
        for chamber in chamber_nodes:
            source=nest
            sink=chamber
            #print(source, sink)
            flow_value, flow_dict = nx.maximum_flow(DG, source, sink, capacity='weight')
            num_edges_traversed = sum([1 for u, v in DG.edges() if flow_dict[u][v] > 0])
            edges_traversed.append(num_edges_traversed)
            total_flow_weight = sum(flow_dict[u][v] for u, edges in flow_dict.items() for v in edges if flow_dict[u][v] > 0)
            #print('FLOW_VALUE=', total_flow_weight)
            CHAM_nodes_passed = []
            passed_nodes=[]
            for u in flow_dict:
                for v in flow_dict[u]:
                    xc=False
                    if flow_dict[u][v] > 0:
                        if u in chamber_nodes and u is not chamber:
                            for n in passed_nodes:
                                if u==n:
                                    xc=True
                            if xc==True:
                                continue
                            CHAM_nodes_passed.append(u)
                            passed_nodes.append(u)
                        if v in chamber_nodes and v is not chamber:
                            for n in passed_nodes:
                                if v==n:
                                    xc=True
                            if xc==True:
                                continue
                            CHAM_nodes_passed.append(v)
                            passed_nodes.append(v)
                            edges_traversed.append(num_edges_traversed)
            num_CHAM_nodes_passed = len(CHAM_nodes_passed)
            ne_ch_flow_value.append(flow_value)
            ne_ch_edges_traversed.append(edges_traversed)
            ne_ch_chambers_traversed.append(num_CHAM_nodes_passed)
            ne_ch_flow_weight.append(total_flow_weight)
    #print(ne_ch_flow_value)
    ne_ch_flow_value_list.append(ne_ch_flow_value)
    ne_ch_edges_traversed_list.append(ne_ch_edges_traversed)
    ne_ch_chambers_traversed_list.append(ne_ch_chambers_traversed)
    ne_ch_flow_weight_list.append(ne_ch_flow_weight)

#now chamber to chamber
ch_ch_flow_value_list=[]
ch_ch_edges_traversed_list=[]
ch_ch_chambers_traversed_list=[]
ch_ch_flow_weight_list=[]

for g in range(0, len(G_list_inverted)):
    ch_ch_flow_value=[]
    ch_ch_edges_traversed=[]
    ch_ch_chambers_traversed=[]
    ch_ch_flow_weight=[]
    G=G_list_inverted[g]
    Gs = sorted(nx.connected_components(G), key=len, reverse=True)
    Gmax = G.subgraph(Gs[0])
    G=Gmax
    DG=G.to_directed()
    attributes = nx.get_node_attributes(DG, 'TYPE')
    #     try:
    chamber_nodes = [node for node, type_value in attributes.items() if type_value == 'CHAM']
    chamber_nodes2=chamber_nodes
    flow_value=[]
    edges_traversed=[]
    chambers_traversed=[]

    completed_paths=[]
    for chamber in chamber_nodes:
        for ch2 in chamber_nodes2:
            if chamber is ch2:
                continue
            completed_paths.append(tuple([ch2, chamber]))
            if tuple([chamber, ch2]) in completed_paths:
                continue
            completed_paths.append(tuple([chamber, ch2]))
            source=chamber
            sink=ch2
            #print(source, sink)
            flow_value, flow_dict = nx.maximum_flow(DG, source, sink, capacity='weight')
            num_edges_traversed = sum([1 for u, v in DG.edges() if flow_dict[u][v] > 0])
            edges_traversed.append(num_edges_traversed)
            total_flow_weight = sum(flow_dict[u][v] for u, edges in flow_dict.items() for v in edges if flow_dict[u][v] > 0)
            #print('FLOW_VALUE=', total_flow_weight)
            CHAM_nodes_passed = []
            passed_nodes=[]
            for u in flow_dict:
                for v in flow_dict[u]:
                    xc=False
                    if flow_dict[u][v] > 0:
                        if u in chamber_nodes and u is not chamber and u is not ch2:
                            for n in passed_nodes:
                                if u==n:
                                    xc=True
                            if xc==True:
                                continue
                            CHAM_nodes_passed.append(u)
                            passed_nodes.append(u)
                        if v in chamber_nodes and v is not chamber and v is not ch2:
                            for n in passed_nodes:
                                if v==n:
                                    xc=True
                            if xc==True:
                                continue
                            CHAM_nodes_passed.append(v)
                            passed_nodes.append(v)
            num_CHAM_nodes_passed = len(CHAM_nodes_passed)
            ch_ch_flow_value.append(flow_value)
            ch_ch_edges_traversed.append(edges_traversed)
            ch_ch_chambers_traversed.append(num_CHAM_nodes_passed)
            ch_ch_flow_weight.append(total_flow_weight)

    ch_ch_flow_value_list.append(ch_ch_flow_value)
    ch_ch_edges_traversed_list.append(ch_ch_edges_traversed)
    ch_ch_chambers_traversed_list.append(ch_ch_chambers_traversed)
    ch_ch_flow_weight_list.append(ch_ch_flow_weight)


#NOW NE -> END
ne_te_flow_value_list=[]
ne_te_edges_traversed_list=[]
ne_te_chambers_traversed_list=[]
ne_te_flow_weight_list=[]

for g in range(0, len(G_list_inverted)):
    ne_te_flow_value=[]
    ne_te_edges_traversed=[]
    ne_te_chambers_traversed=[]
    ne_te_flow_weight=[]
    G=G_list_inverted[g]
    Gs = sorted(nx.connected_components(G), key=len, reverse=True)
    Gmax = G.subgraph(Gs[0])
    G=Gmax
    DG=G.to_directed()
    attributes = nx.get_node_attributes(DG, 'TYPE')
    #     try:
    nest_nodes = [node for node, type_value in attributes.items() if type_value == 'NEST EN']
    end_nodes = [node for node, type_value in attributes.items() if type_value == 'END']
    flow_value=[]
    edges_traversed=[]
    chambers_traversed=[]
    for nest in nest_nodes:
        for end in end_nodes:
            source=nest
            sink=end
            #print(source, sink)
            flow_value, flow_dict = nx.maximum_flow(DG, source, sink, capacity='weight')
            num_edges_traversed = sum([1 for u, v in DG.edges() if flow_dict[u][v] > 0])
            edges_traversed.append(num_edges_traversed)
            total_flow_weight = sum(flow_dict[u][v] for u, edges in flow_dict.items() for v in edges if flow_dict[u][v] > 0)
            #print('FLOW_VALUE=', total_flow_weight)
            CHAM_nodes_passed = []
            passed_nodes=[]
            for u in flow_dict:
                for v in flow_dict[u]:
                    xc=False
                    if flow_dict[u][v] > 0:
                        if u in chamber_nodes:
                            for n in passed_nodes:
                                if u==n:
                                    xc=True
                            if xc==True:
                                continue
                            CHAM_nodes_passed.append(u)
                            passed_nodes.append(u)
                        if v in chamber_nodes:
                            for n in passed_nodes:
                                if v==n:
                                    xc=True
                            if xc==True:
                                continue
                            CHAM_nodes_passed.append(v)
                            passed_nodes.append(v)

            num_CHAM_nodes_passed = len(CHAM_nodes_passed)
            ne_te_flow_value.append(flow_value)
            ne_te_edges_traversed.append(edges_traversed)
            ne_te_chambers_traversed.append(num_CHAM_nodes_passed)
            ne_te_flow_weight.append(total_flow_weight)

    ne_te_flow_value_list.append(ne_te_flow_value)
    ne_te_edges_traversed_list.append(ne_te_edges_traversed)
    ne_te_chambers_traversed_list.append(ne_te_chambers_traversed)
    ne_te_flow_weight_list.append(ne_te_flow_weight)     

ch_te_flow_value_list=[]
ch_te_edges_traversed_list=[]
ch_te_chambers_traversed_list=[]
ch_te_flow_weight_list=[]

for g in range(0, len(G_list_inverted)):
    ch_te_flow_value=[]
    ch_te_edges_traversed=[]
    ch_te_chambers_traversed=[]
    ch_te_flow_weight=[]
    G=G_list_inverted[g]
    Gs = sorted(nx.connected_components(G), key=len, reverse=True)
    Gmax = G.subgraph(Gs[0])
    G=Gmax
    DG=G.to_directed()
    attributes = nx.get_node_attributes(DG, 'TYPE')
    #     try:
    chamber_nodes = [node for node, type_value in attributes.items() if type_value == 'CHAM']
    end_nodes = [node for node, type_value in attributes.items() if type_value == 'END']
    edges_traversed=[]
    chambers_traversed=[]
    for chamber in chamber_nodes:
        for end in end_nodes:
            source=chamber
            sink=end
            #print(source, sink)
            flow_value, flow_dict = nx.maximum_flow(DG, source, sink, capacity='weight')
            num_edges_traversed = sum([1 for u, v in DG.edges() if flow_dict[u][v] > 0])
            edges_traversed.append(num_edges_traversed)
            total_flow_weight = sum(flow_dict[u][v] for u, edges in flow_dict.items() for v in edges if flow_dict[u][v] > 0)
            #print('FLOW_VALUE=', total_flow_weight)
            CHAM_nodes_passed = []
            passed_nodes=[]
            for u in flow_dict:
                for v in flow_dict[u]:
                    xc=False
                    if flow_dict[u][v] > 0:
                        if u in chamber_nodes and u is not chamber:
                            for n in passed_nodes:
                                if u==n:
                                    xc=True
                            if xc==True:
                                continue
                            CHAM_nodes_passed.append(u)
                            passed_nodes.append(u)
                        if v in chamber_nodes and v is not chamber:
                            for n in passed_nodes:
                                if v==n:
                                    xc=True
                            if xc==True:
                                continue
                            CHAM_nodes_passed.append(v)
                            passed_nodes.append(v)

            num_CHAM_nodes_passed = len(CHAM_nodes_passed)
            ch_te_flow_value.append(flow_value)
            ch_te_edges_traversed.append(edges_traversed)
            ch_te_chambers_traversed.append(num_CHAM_nodes_passed)
            ch_te_flow_weight.append(total_flow_weight)

    ch_te_flow_value_list.append(ch_te_flow_value)
    ch_te_edges_traversed_list.append(ch_te_edges_traversed)
    ch_te_chambers_traversed_list.append(ch_te_chambers_traversed)
    ch_te_flow_weight_list.append(ch_te_flow_weight)     




analysis_df['ne_ch_flow_value']=ne_ch_flow_value_list
analysis_df['ne_ch_edges_traversed']=ne_ch_edges_traversed_list
analysis_df['ne_ch_chambers_traversed']=ne_ch_chambers_traversed_list
analysis_df['ne_ch_flow_weight']=ne_ch_flow_weight_list

analysis_df['ch_ch_flow_value']=ch_ch_flow_value_list
analysis_df['ch_ch_edges_traversed']=ch_ch_edges_traversed_list
analysis_df['ch_ch_chambers_traversed']=ch_ch_chambers_traversed_list
analysis_df['ch_ch_flow_weight']=ch_ch_flow_weight_list

analysis_df['ne_te_flow_value']=ne_te_flow_value_list
analysis_df['ne_te_edges_traversed']=ne_te_edges_traversed_list
analysis_df['ne_te_chambers_traversed']=ne_te_chambers_traversed_list
analysis_df['ne_te_flow_weight']=ne_te_flow_weight_list

analysis_df['ch_te_flow_value']=ch_te_flow_value_list
analysis_df['ch_te_edges_traversed']=ch_te_edges_traversed_list
analysis_df['ch_te_chambers_traversed']=ch_te_chambers_traversed_list
analysis_df['ch_te_flow_weight']=ch_te_flow_weight_list



BEGINNING FLOW GRAPH ANALYSIS


This is path length analysis. it runs between chambers and nest entrances (ch-ne), chambers and chambers (ch_ch) and nest-entrances to nest entrances (ne_ne). Looks confusing but short_path is the number of edges in the shortest path. This is repeated using Dijkstra's algorithm, which factors in weights, not edge number (short_paths_list_spatial). For network and dijkstra methods there is also the associated total path length - tot_paths for network; sptl_paths for Dijkstra.

In [45]:
def paths (G, list1, list2):
    chamber_nodes=list1
    nest_nodes=list2
    all_short_paths=[]
    all_short_paths_spatial=[]
    all_tot_paths=[]
    all_spatial_paths=[]
    if len(chamber_nodes)>0:
        for chamber in chamber_nodes:
            shortest_paths=[]
            total_paths=[]
            spatial_paths=[]
            shortest_paths_spatial=[]
            for nest in nest_nodes:
                try:
                    path = nx.shortest_path(G, source=chamber, target=nest)
                    #path_spatial is the shortest euclidean path
                    path_spatial = nx.dijkstra_path(G, source=nest, target=chamber, weight='weight')
                    edges_spatial = [(path_spatial[i], path_spatial[i+1]) for i in range(len(path_spatial)-1)]
                    edge_weights_spatial = [G.get_edge_data(path_spatial[i], path_spatial[i+1])['weight'] for i in range(len(path_spatial) - 1)]
                    edge_num_spatial=len(edges_spatial)
                    path_dist_spatial=np.sum(edge_weights_spatial)
                    shortest_paths_spatial.append(edge_num_spatial)
                    spatial_paths.append(path_dist_spatial)#print("Shortest path between {} and {}: {}".format(chamber, nest, path))
                    edges = [(path[i], path[i+1]) for i in range(len(path)-1)]
                    edge_weights = [G.get_edge_data(path[i], path[i+1])['weight'] for i in range(len(path) - 1)]
                    edge_num=len(edges)
                    path_dist=np.sum(edge_weights)
                    shortest_paths.append(edge_num)
                    total_paths.append(path_dist)
                    
                except nx.NetworkXNoPath:
                    print("No path found between {} and {}".format(chamber, nest))
            all_short_paths.extend(shortest_paths)
            all_tot_paths.extend(total_paths)
            all_spatial_paths.extend(spatial_paths)
            all_short_paths_spatial.extend(shortest_paths_spatial)


    else:
        all_short_paths=np.nan
        all_tot_paths=np.nan
        all_sptl_paths=np.nan

    return all_short_paths, all_tot_paths, all_spatial_paths, all_short_paths_spatial

This calculates path lengths for an attribute between itself.

In [33]:
#now calculate CHAMBER TO CHAMBER PATHS
def paths_same_attribute (G, list1):
    chamber_nodes=list1
    chamber_node2=list1
    chch_all_short_paths_list=[]
    chch_all_tot_paths_list=[]
    chch_all_sptl_paths_list=[]
    chch_all_short_paths_list_spatial=[]
    if len(chamber_nodes)>0:
        chamber_nodes2=chamber_nodes
        completed_paths=[]
        for chamber in chamber_nodes:
            shortest_paths=[]
            total_paths=[]
            spatial_paths=[]
            shortest_paths_spatial=[]
            chch_all_short_paths_spatial=[]
            for ch2 in chamber_nodes2:
                if chamber is not ch2:
                    try:
                        completed_path = nx.shortest_path(G, source=ch2, target=chamber)
                        if tuple(completed_path) in completed_paths:
                            continue
                        completed_paths.append(tuple(completed_path))

                        path = nx.shortest_path(G, source=chamber, target=ch2)
                        completed_paths.append(tuple(path))
                        #path_spatial is the shortest euclidean path
                        path_spatial = nx.dijkstra_path(G, source=chamber, target=ch2, weight='weight')
                        edges_spatial = [(path_spatial[i], path_spatial[i+1]) for i in range(len(path_spatial)-1)]
                        edge_weights_spatial = [G.get_edge_data(path_spatial[i], path_spatial[i+1])['weight'] for i in range(len(path_spatial) - 1)]
                        edge_num_spatial=len(edges_spatial)
                        path_dist_spatial=np.sum(edge_weights_spatial)
                        shortest_paths_spatial.append(edge_num_spatial)
                        spatial_paths.append(path_dist_spatial)#print("Shortest path between {} and {}: {}".format(chamber, nest, path))
                        edges = [(path[i], path[i+1]) for i in range(len(path)-1)]
                        edge_weights = [G.get_edge_data(path[i], path[i+1])['weight'] for i in range(len(path) - 1)]
                        edge_num=len(edges)
                        path_dist=np.sum(edge_weights)
                        shortest_paths.append(edge_num)
                        total_paths.append(path_dist)
                        spatial_paths.append(path_dist_spatial)

                    except nx.NetworkXNoPath:
                        print("No path found between {} and {}".format(chamber, ch2))
            chch_all_short_paths_list.extend(shortest_paths)
            chch_all_tot_paths_list.extend(total_paths)
            chch_all_sptl_paths_list.extend(spatial_paths)
            chch_all_short_paths_list_spatial.extend(shortest_paths_spatial)
    else:
        chch_all_short_paths=np.nan
        chch_all_tot_paths=np.nan
        chch_all_sptl_paths=np.nan
        chch_all_short_paths_spatial=np.nan
    return chch_all_short_paths_list, chch_all_tot_paths_list, chch_all_sptl_paths_list, chch_all_short_paths_list_spatial

In [46]:
#junction-junction paths
j_all_short_paths_list=[]
j_all_tot_paths_list=[]
j_all_sptl_paths_list=[]
j_all_short_paths_list_spatial=[]

#ch-ch paths
chch_all_short_paths_list=[]
chch_all_tot_paths_list=[]
chch_all_sptl_paths_list=[]
chch_all_short_paths_list_spatial=[]

#j-ch paths
jch_all_short_paths_list=[]
jch_all_tot_paths_list=[]
jch_all_sptl_paths_list=[]
jch_all_short_paths_list_spatial=[]

#ne-ch paths
nech_all_short_paths_list=[]
nech_all_tot_paths_list=[]
nech_all_sptl_paths_list=[]
nech_all_short_paths_list_spatial=[]

NECH_COMBINED_PATH_LIST=[]
CH_COMBINED_PATH_LIST=[]

for g in range(len(G_list)):    
    G=G_list[g]
    attributes = nx.get_node_attributes(G, 'TYPE')
    #     try:
    nest_nodes = [node for node, type_value in attributes.items() if type_value == 'NEST EN']
    chamber_nodes = [node for node, type_value in attributes.items() if type_value == 'CHAM']
    junction_nodes = [node for node, type_value in attributes.items() if type_value == 'JUNC']
    end_nodes = [node for node, type_value in attributes.items() if type_value == 'END']
    
    j_all_short_paths, j_all_tot_paths, j_all_sptl_paths, j_all_short_paths_spatial=paths_same_attribute(G, junction_nodes)
    ch_all_short_paths, ch_all_tot_paths, ch_all_sptl_paths, ch_all_short_paths_spatial=paths_same_attribute(G, chamber_nodes)
    jch_all_short_paths, jch_all_tot_paths, jch_all_sptl_paths, jch_all_short_paths_spatial=paths(G, chamber_nodes, junction_nodes)
    nech_all_short_paths, nech_all_tot_paths, nech_all_sptl_paths, nech_all_short_paths_spatial=paths(G, nest_nodes, chamber_nodes)
    #nej_all_short_paths_list, nej_all_tot_paths_list, nej_all_sptl_paths_list, nej_all_short_paths_list_spatial=paths(G, nest_nodes, junction_nodes)
    nech_combined=[]
    for i in range(len(nech_all_tot_paths)):
    # Calculate the mean of the corresponding elements at each index
        mean_val = (nech_all_tot_paths[i] + nech_all_sptl_paths[i]) / 2
        nech_combined.append(mean_val)
        # Append the mean value to the list
    NECH_COMBINED_PATH_LIST.append(nech_combined)
    
    ch_combined=[]
    for i in range(len(ch_all_tot_paths)):
    # Calculate the mean of the corresponding elements at each index
        mean_val = (ch_all_tot_paths[i] + ch_all_sptl_paths[i]) / 2
        ch_combined.append(mean_val)
        # Append the mean value to the list
    CH_COMBINED_PATH_LIST.append(ch_combined)

    
    
    j_all_short_paths_list.append(j_all_short_paths)
    j_all_tot_paths_list.append(j_all_tot_paths)
    j_all_sptl_paths_list.append(j_all_sptl_paths)
    j_all_short_paths_list_spatial.append(j_all_short_paths_spatial)

    #ch-ch paths
    chch_all_short_paths_list.append(ch_all_short_paths)
    chch_all_tot_paths_list.append(ch_all_tot_paths)
    chch_all_sptl_paths_list.append(ch_all_sptl_paths)
    chch_all_short_paths_list_spatial.append(ch_all_short_paths_spatial)

    #j-ch paths
    jch_all_short_paths_list.append(jch_all_short_paths)
    jch_all_tot_paths_list.append(jch_all_tot_paths)
    jch_all_sptl_paths_list.append(jch_all_sptl_paths)
    jch_all_short_paths_list_spatial.append(jch_all_short_paths_spatial)

    #ne-ch paths
    nech_all_short_paths_list.append(nech_all_short_paths)
    nech_all_tot_paths_list.append(nech_all_tot_paths)
    nech_all_sptl_paths_list.append(nech_all_sptl_paths)
    nech_all_short_paths_list_spatial.append(nech_all_short_paths_spatial)
    
analysis_df['j_all_short_paths_list']=j_all_short_paths_list
analysis_df['j_all_tot_paths_list']=j_all_tot_paths_list
analysis_df['j_all_sptl_paths_list']=j_all_sptl_paths_list
analysis_df['j_all_short_paths_list_spatial']=j_all_short_paths_list_spatial

analysis_df['chch_all_short_paths_list']=chch_all_short_paths_list
analysis_df['chch_all_tot_paths_list']=chch_all_tot_paths_list
analysis_df['chch_all_sptl_paths_list']=chch_all_sptl_paths_list
analysis_df['chch_all_short_paths_list_spatial']=chch_all_short_paths_list_spatial

analysis_df['jch_all_short_paths_list']=jch_all_short_paths_list
analysis_df['jch_all_tot_paths_list']=jch_all_tot_paths_list
analysis_df['jch_all_sptl_paths_list']=jch_all_sptl_paths_list
analysis_df['jch_all_short_paths_list_spatial']=jch_all_short_paths_list_spatial

analysis_df['nech_all_short_paths_list']=nech_all_short_paths_list
analysis_df['nech_all_tot_paths_list']=nech_all_tot_paths_list
analysis_df['nech_all_sptl_paths_list']=nech_all_sptl_paths_list
analysis_df['nech_all_short_paths_list_spatial']=nech_all_short_paths_list_spatial


analysis_df['nech_combined_paths_list']=NECH_COMBINED_PATH_LIST
analysis_df['ch_combined_paths_list']=CH_COMBINED_PATH_LIST
    

This calculates the shortest paths between all pairs of nodes in a g.. It does everything twice, using edge number and dijkstra distances. It returns the diamete in edges first, then the associated path, then for dijkstra it gives edge number between each pair then summed edge wights, then for network measure it does the  same.  This has been validated with the networkx out-of-the box measure.

In [35]:
max_spatial_edges_list=[]
max_edge_diameter_list=[] 
max_spatial_diameter_list=[]
max_edge_spatial_path_list=[]
spatial_edges_list=[]
spatial_lengths_list=[]
edge_paths_list=[]
edge_length_list=[]
for G in G_list:
    max_spatial_edges,max_edge_diameter, max_spatial_diameter,max_edge_spatial_path, spatial_edges, spatial_lengths, edge_paths, edge_lengths=D3.calculate_diameter(G)
    print('edge diam spatial/path=', max_spatial_edges, max_edge_diameter)
    print('spatial diam spatial/path=',max_spatial_diameter,max_edge_spatial_path)
    
    max_spatial_edges_list.append(max_spatial_edges)
    max_edge_diameter_list.append(max_edge_diameter)
    max_spatial_diameter_list.append(max_spatial_diameter)
    max_edge_spatial_path_list.append(max_edge_spatial_path)
    spatial_edges_list.append(spatial_edges)
    spatial_lengths_list.append(spatial_lengths)
    edge_paths_list.append(edge_paths)
    edge_length_list.append(edge_lengths)
    
analysis_df['max_spatial_edges_list']=max_spatial_edges_list
analysis_df['max_edge_diameter_list']=max_edge_diameter_list
analysis_df['max_spatial_diameter_list']=max_spatial_diameter_list
analysis_df['max_edge_spatial_path_list']=max_edge_spatial_path_list
analysis_df['spatial_edges_list']=spatial_edges_list
analysis_df['spatial_lengths_list']=spatial_lengths_list
analysis_df['edge_paths_list']=edge_paths_list
analysis_df['edge_length_list']=edge_length_list

edge diam spatial/path= 7 7
spatial diam spatial/path= 65.53965854713262 57.245782006405435
edge diam spatial/path= 6 10
spatial diam spatial/path= 1010.2993926089418 1040.510688157238
edge diam spatial/path= 13 10
spatial diam spatial/path= 745.1136359179777 387.1018609015524
edge diam spatial/path= 10 8
spatial diam spatial/path= 1426.1107790706956 1291.9608272249236
edge diam spatial/path= 12 10
spatial diam spatial/path= 3289.7308282623894 557.0168499916592
edge diam spatial/path= 7 10
spatial diam spatial/path= 412.13536770467726 295.0081863193967
edge diam spatial/path= 7 14
spatial diam spatial/path= 3450.566203915874 3175.231671875835
edge diam spatial/path= 15 14
spatial diam spatial/path= 323.0245431070769 478.58237682128646
edge diam spatial/path= 16 16
spatial diam spatial/path= 816.8021475927222 1467.072926194592
edge diam spatial/path= 23 16
spatial diam spatial/path= 630.5963405558672 616.5549357347261
edge diam spatial/path= 3 11
spatial diam spatial/path= 19604.7328485

edge diam spatial/path= 10 10
spatial diam spatial/path= 336.95726048139966 218.50741493218985
edge diam spatial/path= 4 4
spatial diam spatial/path= 89.54283395337136 60.575078772145005
edge diam spatial/path= 7 7
spatial diam spatial/path= 821.8740122071147 533.8517114163669
edge diam spatial/path= 7 8
spatial diam spatial/path= 263.5516734797048 239.42128466798826
edge diam spatial/path= 13 9
spatial diam spatial/path= 609.1941994438001 473.0026572837448
edge diam spatial/path= 9 9
spatial diam spatial/path= 426.14050082898586 923.0414576723903
edge diam spatial/path= 7 7
spatial diam spatial/path= 233.52436607441155 122.84380488915063
edge diam spatial/path= 10 10
spatial diam spatial/path= 302.10567840458384 302.10567840458384
edge diam spatial/path= 8 10
spatial diam spatial/path= 377.5676262942638 334.5520282040434
edge diam spatial/path= 5 12
spatial diam spatial/path= 1044.3848466682375 220.81489829584834
edge diam spatial/path= 8 12
spatial diam spatial/path= 583.378280820013

Main graph analysis:

In [36]:

#uninverted measures


spatial_diameter_list=[]
max_edge_diameter_list=[]
max_edge_path_list =[]
connected_components_list=[]


deg_cen_list = []
mean_deg_cen_list = []
std_deg_cen_list = []
clust_coeff_list = []
mean_clust_list = []
std_clust_list = []
trans_list = []

square_clustering_list=[]
mean_square_clustering_list = []
std_square_clustering_list = []
triangles_list = []
density_list = []

closeness_centrality_spatial_list=[]
mean_closeness_centrality_spatial_list = []
std_closeness_centrality_spatial_list = []

degree_list=[]
mean_degree_list = []
std_degree_list = []

degree_centrality_list = []
mean_degree_centrality_list = []
std_degree_centrality_list = []

cham_degree_list = []
mean_chamber_degree_list = []
std_chamber_degree_list = []


cham_degree_centrality_list = []
mean_cham_deg_centrality_list = []
std_cham_deg_centrality_list = []

cham_closeness_spatial_list = []
mean_chamber_closeness_centrality_spatial_list = []
std_chamber_closeness_centrality_spatial_list = []

cham_clustering_list=[]
mean_cham_clustering_list=[]
std_cham_clustering_list=[]

junc_degree_list = []
mean_junc_degree_list = []
std_junc_degree_list = []

junc_degree_centrality_list = []
mean_junc_degree_centrality_list = []
std_junc_degree_centrality_list = []

junc_closeness_spatial_list = []
mean_junc_closeness_centrality_spatial_list = []
std_junc_closeness_centrality_spatial_list = []

junc_clustering_list=[]
mean_junc_clustering_list=[]
std_cham_clustering_list=[]

ne_degree_list = []
mean_ne_degree_list = []
std_ne_degree_list = []

ne_degree_centrality_list = []
mean_ne_deg_centrality_list = []
std_ne_deg_centrality_list = []

ne_closeness_spatial_list = []
mean_ne_closeness_centrality_spatial_list = []
std_ne_closeness_centrality_spatial_list = []

junc_clustering_list=[]
mean_junc_clustering_list=[]
std_junc_clustering_list=[]

#spatial_path_list=[]
#edge_path_list=[]

# nct_grp_bet_cen_list=[]
# nct_grp_clo_cen_list=[]
# nct_grp_deg_cen_list=[]

# nech_grp_bet_cen_list=[]
# nech_grp_clo_cen_list=[]
# nech_grp_deg_cen_list=[]
# #just ch
# ch_grp_bet_cen_list=[]
# ch_grp_clo_cen_list=[]
# ch_grp_deg_cen_list=[]

for g in range(0,len(G_list)):
    G=G_list[g]
    Gs = sorted(nx.connected_components(G), key=len, reverse=True)
    Gmax = G.subgraph(Gs[0])
    G=Gmax
    attributes = nx.get_node_attributes(G, 'TYPE')
    ALL_JUNC_NUMS = [node for node, type_value in attributes.items() if type_value == 'JUNC']
    ALL_CHAM_NUMS = [node for node, type_value in attributes.items() if type_value == 'CHAM']
    ALL_NE_NUMS = [node for node, type_value in attributes.items() if type_value == 'NE']
    END = [node for node, type_value in attributes.items() if type_value == 'END']
    JC=list(ALL_JUNC_NUMS+ALL_CHAM_NUMS)
    JN=list(ALL_JUNC_NUMS+ALL_NE_NUMS)
    JE=list(ALL_JUNC_NUMS+END)
    JJ=list(ALL_JUNC_NUMS+ALL_JUNC_NUMS)
    

    CNT=list(set(ALL_CHAM_NUMS + ALL_NE_NUMS + END)) 
    CN=list(set(ALL_CHAM_NUMS + ALL_NE_NUMS))
    CH1=list(set(ALL_CHAM_NUMS))


#     try:
#         nct_grp_bet_cen=nx.group_betweenness_centrality(G, CNT, normalized=False, weight=True, endpoints=True)
#     except ZeroDivisionError:
#         nct_grp_bet_cen=np.nan
#     nct_grp_clo_cen=nx.group_closeness_centrality(G, CNT, weight=True)
#     nct_grp_deg_cen=nx.group_degree_centrality(G, CNT)


#     try:
#         nech_grp_bet_cen=nx.group_betweenness_centrality(G, CN, normalized=False, weight=True, endpoints=True)
#     except ZeroDivisionError:
#         nech_grp_bet_cen=np.nan    
#     nech_grp_clo_cen=nx.group_closeness_centrality(G, CN, weight=True)
#     print(CN)
#     nech_grp_deg_cen=nx.group_degree_centrality(G, CN)
#     #just ch
#     try:
#         ch_grp_bet_cen=nx.group_betweenness_centrality(G, CH1, normalized=False, weight=True, endpoints=True)
#     except ZeroDivisionError:
#         ch_grp_bet_cen=np.nan
#     ch_grp_clo_cen=nx.group_closeness_centrality(G, CH1, weight=True)
#     ch_grp_deg_cen=nx.group_degree_centrality(G, CH1)

#     nct_grp_bet_cen_list.append(nct_grp_bet_cen)
#     nct_grp_clo_cen_list.append(nct_grp_clo_cen)
#     nct_grp_deg_cen_list.append(nct_grp_deg_cen)

#     nech_grp_bet_cen_list.append(nech_grp_bet_cen)
#     nech_grp_clo_cen_list.append(nech_grp_clo_cen)
#     nech_grp_deg_cen_list.append(nech_grp_deg_cen)
#     #just ch
#     ch_grp_bet_cen_list.append(ch_grp_bet_cen)
#     ch_grp_clo_cen_list.append(ch_grp_clo_cen)
#     ch_grp_deg_cen_list.append(ch_grp_deg_cen)
#     #get main connected component

    #G=Gmax

    deg_cen=nx.degree_centrality(G)
    deg_cen_l=list(deg_cen.values())
    np_deg_cen=np.array(deg_cen_l)
    mean_deg_cen=np.mean(np_deg_cen)
    std_deg_cen=np.std(np_deg_cen)
    print("mean degree centrality=",mean_deg_cen)
    print("std dev degree centrality=",std_deg_cen)

    clust_coeff = nx.average_clustering(Gmax)
    clust = nx.clustering(Gmax)
    clust1=list(clust.values())
    clust1=np.array(clust1)
    mean_clust=np.mean(clust1)
    std_clust=np.std(clust1)
    print("clustering coefficient=",clust_coeff)
    print("std dev clustering=",std_clust)
    trans= nx.transitivity(Gmax)
    print("transitivity",trans)
    square_clustering = nx.square_clustering(G)
    mean_square_clustering = sum(square_clustering.values())/len(square_clustering)
    print("square clustering",mean_square_clustering)
    square_clustering_values = list(square_clustering.values())
    std_square_clustering = np.std(square_clustering_values)
    print("std dev square clustering",std_square_clustering)
    triangles = sum(nx.triangles(G).values()) / 3
    print("triangles=", triangles)
    density=nx.density(G)

    closeness_centrality_spatial = nx.closeness_centrality(G, distance='weight')
    mean_closeness_centrality_spatial = sum(closeness_centrality_spatial.values())/len(closeness_centrality_spatial)
    print("closeness_centrality_spatial",mean_closeness_centrality_spatial)
    closeness_centrality_spatial_values = list(closeness_centrality_spatial.values())
    std_closeness_centrality_spatial = np.std(closeness_centrality_spatial_values)
    degree=nx.degree(G)
    degree = {node: d for node, d in degree}
    mean_degree= sum(degree.values())/len(degree)
    degree_values = list(degree.values())
    std_degree = np.std(degree_values)

    cham_degree=[]
    cham_closeness_spatial=[]
    cham_degree_centrality=[]
    cham_clustering=[]
    for cham in ALL_CHAM_NUMS:
        try:
        #print(len(degree), nx.number_of_nodes(G))
#         print(g, 'CH=', ALL_CHAM_NUMS[g])
#         print(nx.nodes(G))
            ch_deg=degree[cham]

            cham_degree.append(ch_deg)
            cham_close_spatial=closeness_centrality_spatial[cham]
            cham_closeness_spatial.append(cham_close_spatial)
            cham_deg_cen=deg_cen[cham]
            cham_degree_centrality.append(cham_deg_cen)
            try:
                cham_clust=clust[cham]
            except KeyError:
                cham_clust=np.nan
            cham_clustering.append(cham_clust)
        except KeyError:
            continue


    mean_cham_deg_centrality=np.mean(cham_degree_centrality)
    std_cham_deg_centrality=np.std(cham_degree_centrality)
    mean_chamber_degree=np.mean(cham_degree)
    std_chamber_degree=np.std(cham_degree)
    mean_chamber_closeness_centrality_spatial=np.mean(cham_closeness_spatial)
    std_chamber_closeness_centrality_spatial=np.std(cham_closeness_spatial)
    mean_cham_clustering=np.nanmean(cham_clustering)
    std_cham_clustering=np.nanstd(cham_clustering)

    junc_degree=[]
    junc_degree_centrality=[]
    junc_closeness_spatial=[]
    junc_clustering=[]
    for junc in ALL_JUNC_NUMS:
        try:
            ju_deg=degree[junc]
            junc_degree.append(ju_deg)
            junc_close_spatial=closeness_centrality_spatial[junc]
            junc_closeness_spatial.append(junc_close_spatial)
            ju_deg_cen=deg_cen[junc]
            junc_degree_centrality.append(ju_deg_cen)
            try:
                junc_clust=clust[junc]
            except KeyError:
                junc_clust=np.nan
            junc_clustering.append(junc_clust)
        except KeyError:
            continue

    mean_junc_deg_centrality=np.mean(junc_degree_centrality)
    std_junc_deg_centrality=np.std(junc_degree_centrality)
    mean_junc_degree=np.mean(junc_degree)
    std_junc_degree=np.std(junc_degree)
    mean_junc_closeness_centrality_spatial=np.mean(junc_closeness_spatial)
    std_junc_closeness_centrality_spatial=np.std(junc_closeness_spatial)
    mean_junc_clustering=np.nanmean(junc_clustering)
    std_junc_clustering=np.nanstd(junc_clustering)

    ne_degree=[]
    ne_degree_centrality=[]
    ne_closeness=[]
    ne_closeness_spatial=[]
#     ne_clustering=[]
    for ne in ALL_NE_NUMS:
        try:
            ne_deg=degree[ne]
            ne_degree.append(ne_deg)
            ne_close_spatial=closeness_centrality_spatial[ne]
            ne_closeness_spatial.append(ne_close_spatial)
            ne_deg_cen=deg_cen[ne]
            ne_degree_centrality.append(ne_deg_cen)
        except KeyError:
            continue
#         try:
#             ne_clust=clust[ne]
#         except KeyError:
#             ne_clust=np.nan
#         ne_clustering.append(ne_clust)

    mean_ne_deg_centrality=np.nanmean(ne_degree_centrality)
    std_ne_deg_centrality=np.nanstd(ne_degree_centrality)
    mean_ne_degree=np.nanmean(ne_degree)
    std_ne_degree=np.nanstd(ne_degree)
    mean_ne_closeness_centrality=np.nanmean(ne_closeness)
    std_ne_closeness_centrality=np.nanstd(ne_closeness)
    mean_ne_closeness_centrality_spatial=np.nanmean(ne_closeness_spatial)
    std_ne_closeness_centrality_spatial=np.nanstd(ne_closeness_spatial)
#     mean_ne_clustering=np.mean(ne_clustering)
#     std_ne_clustering=np.std(ne_clustering)


    deg_cen_list.append(deg_cen_l)
    mean_deg_cen_list.append(mean_deg_cen)
    std_deg_cen_list.append(std_deg_cen)
    clust_coeff_list.append(clust1)
    mean_clust_list.append(mean_clust)
    std_clust_list.append(std_clust)
    trans_list.append(trans)

    square_clustering_list.append(square_clustering_values)
    mean_square_clustering_list.append(mean_square_clustering)
    std_square_clustering_list.append(std_square_clustering)
    triangles_list.append(triangles)
    density_list.append(density)

    closeness_centrality_spatial_list.append(closeness_centrality_spatial)
    mean_closeness_centrality_spatial_list.append(mean_closeness_centrality_spatial)
    std_closeness_centrality_spatial_list.append(std_closeness_centrality_spatial)

    degree_list.append(degree_values)
    mean_degree_list.append(mean_degree)
    std_degree_list.append(std_degree)

    cham_degree_list.append(cham_degree)
    mean_chamber_degree_list.append(mean_chamber_degree)
    std_chamber_degree_list.append(std_chamber_degree)

    cham_degree_centrality_list.append(cham_degree_centrality)
    mean_cham_deg_centrality_list.append(mean_cham_deg_centrality)
    std_cham_deg_centrality_list.append(std_cham_deg_centrality)

    cham_closeness_spatial_list.append(cham_closeness_spatial)
    mean_chamber_closeness_centrality_spatial_list.append(mean_chamber_closeness_centrality_spatial)
    std_chamber_closeness_centrality_spatial_list.append(std_chamber_closeness_centrality_spatial)

    cham_clustering_list.append(cham_clustering)
    mean_cham_clustering_list.append(mean_cham_clustering)
    std_cham_clustering_list.append(std_cham_clustering)

    junc_degree_list.append(junc_degree)
    mean_junc_degree_list.append(mean_junc_degree)
    std_junc_degree_list.append(std_junc_degree)

    junc_degree_centrality_list.append(junc_degree_centrality)
    mean_junc_degree_centrality_list.append(mean_junc_deg_centrality)
    std_junc_degree_centrality_list.append(std_junc_deg_centrality)

    junc_closeness_spatial_list.append(junc_closeness_spatial)
    mean_junc_closeness_centrality_spatial_list.append(mean_junc_closeness_centrality_spatial)
    std_junc_closeness_centrality_spatial_list.append(std_junc_closeness_centrality_spatial)

    junc_clustering_list.append(junc_clustering)
    mean_junc_clustering_list.append(mean_junc_clustering)
    std_junc_clustering_list.append(std_junc_clustering)

    ne_degree_list.append(ne_degree)
    mean_ne_degree_list.append(mean_ne_degree)
    std_ne_degree_list.append(std_ne_degree)

    ne_degree_centrality_list.append(ne_degree_centrality)
    mean_ne_deg_centrality_list.append(mean_ne_deg_centrality)
    std_ne_deg_centrality_list.append(std_ne_deg_centrality)

    ne_closeness_spatial_list.append(ne_closeness_spatial)
    mean_ne_closeness_centrality_spatial_list.append(mean_ne_closeness_centrality_spatial)
    std_ne_closeness_centrality_spatial_list.append(std_ne_closeness_centrality_spatial)

#     ne_clustering_list.append(ne_clustering)
#     mean_ne_clustering_list.append(mean_ne_clustering)
#     std_ne_clustering_list.append(std_ne_clustering)

#analysis_df['spatial_path_list'] = spatial_path_list
#analysis_df['sedge_path_list'] = edge_path_list


analysis_df['degree_centrality'] = deg_cen_list
analysis_df['mean_degree_centrality'] = mean_deg_cen_list
analysis_df['std_degree_centrality'] = std_deg_cen_list

analysis_df['clustering_coefficient'] = clust_coeff_list
analysis_df['mean_clustering'] = mean_clust_list
analysis_df['std_clustering'] = std_clust_list
analysis_df['transitivity'] = trans_list

analysis_df['square_clustering'] = square_clustering_list
analysis_df['mean_square_clustering'] = mean_square_clustering_list
analysis_df['std_square_clustering'] = std_square_clustering_list
analysis_df['triangles'] = triangles_list
analysis_df['density'] = density_list

analysis_df['closeness_centrality_spatial'] = closeness_centrality_spatial_list
analysis_df['mean_closeness_centrality_spatial'] = mean_closeness_centrality_spatial_list
analysis_df['std_closeness_centrality_spatial'] = std_closeness_centrality_spatial_list

analysis_df['degree'] = degree_list
analysis_df['mean_degree'] = mean_degree_list
analysis_df['std_degree'] = std_degree_list

# analysis_df['degree_centrality'] = degree_centrality_list
# analysis_df['mean_degree_centrality'] = mean_degree_centrality_list
# analysis_df['std_degree_centrality'] = std_degree_centrality_list

analysis_df['chamber_degree'] = cham_degree_list
analysis_df['mean_chamber_degree'] = mean_chamber_degree_list
analysis_df['std_chamber_degree'] = std_chamber_degree_list

analysis_df['chamber_degree_centrality'] = cham_degree_centrality_list
analysis_df['mean_chamber_degree_centrality'] = mean_cham_deg_centrality_list
analysis_df['std_chamber_degree_centrality'] = std_cham_deg_centrality_list

analysis_df['chamber_closeness_spatial'] = cham_closeness_spatial_list
analysis_df['mean_chamber_closeness_centrality_spatial'] = mean_chamber_closeness_centrality_spatial_list
analysis_df['std_chamber_closeness_centrality_spatial'] = std_chamber_closeness_centrality_spatial_list

analysis_df['junction_degree'] = junc_degree_list
analysis_df['mean_junction_degree'] = mean_junc_degree_list
analysis_df['std_junction_degree'] = std_junc_degree_list

analysis_df['junction_degree_centrality'] = junc_degree_centrality_list
analysis_df['mean_junction_degree_centrality'] = mean_junc_degree_centrality_list
analysis_df['std_junction_degree_centrality'] = std_junc_degree_centrality_list

analysis_df['junction_closeness_spatial'] = junc_closeness_spatial_list
analysis_df['mean_junction_closeness_centrality_spatial'] = mean_junc_closeness_centrality_spatial_list
analysis_df['std_junction_closeness_centrality_spatial'] = std_junc_closeness_centrality_spatial_list

analysis_df['ne_degree'] = ne_degree_list
analysis_df['mean_ne_degree'] = mean_ne_degree_list
analysis_df['std_ne_degree'] = std_ne_degree_list

analysis_df['ne_degree_centrality'] = ne_degree_centrality_list
analysis_df['mean_ne_degree_centrality'] = mean_ne_deg_centrality_list
analysis_df['std_ne_degree_centrality'] = std_ne_deg_centrality_list

analysis_df['ne_closeness_spatial'] = ne_closeness_spatial_list
analysis_df['mean_ne_closeness_centrality_spatial'] = mean_ne_closeness_centrality_spatial_list
analysis_df['std_ne_closeness_centrality_spatial'] = std_ne_closeness_centrality_spatial_list

# analysis_df['nct_grp_bet_cen_list']=nct_grp_bet_cen_list
# analysis_df['nct_grp_clo_cen_list']=nct_grp_clo_cen_list
# analysis_df['nct_grp_deg_cen_list']=nct_grp_deg_cen_list

# analysis_df['nech_grp_bet_cen_list']=nech_grp_bet_cen_list
# analysis_df['nech_grp_clo_cen_list']=nech_grp_clo_cen_list
# analysis_df['nech_grp_deg_cen_list']=nech_grp_deg_cen_list

# analysis_df['ch_grp_bet_cen_list']=ch_grp_bet_cen_list
# analysis_df['ch_grp_clo_cen_list']=ch_grp_clo_cen_list
# analysis_df['ch_grp_deg_cen_list']=ch_grp_deg_cen_list
#now add al these measure to a list


#betweenness is uninverted. flow is inverted.
#retain the main CCP
betweenness_centrality_list=[] #bet_cen_values
mean_betweenness_centrality=[]
std_betweenness_centrality=[]

degree_assortativity=[]
global_assortativity=[]
cham_assortativity=[]
junc_assortativity=[]
ne_assortativity=[]
end_assortativity=[]

efficiency=[]

communities=[]
modularities=[]

flow_centrality_list=[]
mean_flow_centralities=[]
std_flow_centralities=[]

spectral_gap=[]

harmonic_centrality_list=[]
mean_harmonic_centralities=[]
std_harmonic_centralities=[]

cham_flow_centrality_list=[]
cham_mean_flow_centralities=[]
cham_std_flow_centralities=[]

cham_betweenness_centrality_list=[]
cham_mean_betweenness_centrality=[]
cham_std_betweenness_centrality=[]

junc_flow_centrality_list=[]
junc_mean_flow_centralities=[]
junc_std_flow_centralities=[]

junc_betweenness_centrality_list=[]
junc_mean_betweenness_centrality=[]
junc_std_betweenness_centrality=[]

ne_flow_centrality_list=[]
ne_mean_flow_centrality=[]
ne_std_flow_centrality=[]

ne_betweenness_centrality_list=[]
ne_mean_betweenness_centrality=[]
ne_std_betweenness_centrality=[]

junction_degree_assortativity=[]
chamber_degree_assortativity=[]
JC_assortativity=[]
JN_assortativity=[]
JE_assortativity=[]

for g in range(0,len(G_list_inverted)):
    G=G_list_inverted[g]
    Gs = sorted(nx.connected_components(G), key=len, reverse=True)
    Gmax = G.subgraph(Gs[0])
    G=Gmax
    #uninverted for betweenness
    G2=G_list[g]
    Gs2 = sorted(nx.connected_components(G2), key=len, reverse=True)
    Gmax2 = G2.subgraph(Gs2[0])
    G2=Gmax2
    #G=Gmax
    bet_cen = nx.betweenness_centrality(G2)
    mean_bc = sum(bet_cen.values())/len(bet_cen)
    mean_betweenness_centrality.append(mean_bc)
    bet_cen_values = list(bet_cen.values())
    betweenness_centrality_list.append(bet_cen_values)
    std_bc = np.std(bet_cen_values)
    std_betweenness_centrality.append(std_bc)
    print("mean betweenness centrality=",mean_bc)
    print("std dev betweenness centrality=",std_bc)
    
    attributes = nx.get_node_attributes(G, 'TYPE')
    ALL_JUNC_NUMS = [node for node, type_value in attributes.items() if type_value == 'JUNC']
    ALL_CHAM_NUMS = [node for node, type_value in attributes.items() if type_value == 'CHAM']
    ALL_NE_NUMS = [node for node, type_value in attributes.items() if type_value == 'NE']
    END = [node for node, type_value in attributes.items() if type_value == 'END']
    JC=list(ALL_JUNC_NUMS+ALL_CHAM_NUMS)
    print(JC)
    JN=list(ALL_JUNC_NUMS+ALL_NE_NUMS)
    JE=list(ALL_JUNC_NUMS+END)
    JJ=list(ALL_JUNC_NUMS+ALL_JUNC_NUMS)
#     JC=list(JUNC+CHAM)
#     JC=list(JUNC+CHAM)
    JC_assortativity.append(nx.attribute_assortativity_coefficient(G, 'TYPE', nodes=JC))
    JN_assortativity.append(nx.attribute_assortativity_coefficient(G, 'TYPE', nodes=JN))
    JE_assortativity.append(nx.attribute_assortativity_coefficient(G, 'TYPE', nodes=JE))
    

    degree_assortativity.append(nx.degree_assortativity_coefficient(G))
    junction_degree_assortativity.append(nx.degree_assortativity_coefficient(G, nodes= ALL_JUNC_NUMS))
    chamber_degree_assortativity.append(nx.degree_assortativity_coefficient(G, nodes=ALL_CHAM_NUMS))
    
    global_assortativity.append(nx.attribute_assortativity_coefficient(G, 'TYPE'))
    cham_assortativity.append(nx.attribute_assortativity_coefficient(G, 'CHAM'))
    ne_assortativity.append(nx.attribute_assortativity_coefficient(G, 'NE'))
    junc_assortativity.append(nx.attribute_assortativity_coefficient(G, 'JUNC'))
    end_assortativity.append(nx.attribute_assortativity_coefficient(G, 'END'))
    efficiency.append(nx.global_efficiency(Gmax))
    comms = community.best_partition(G)
    comms1=[*set(list(comms.values()))]
    community_num=len(comms1)
    communities.append(community_num)
    print("community_num=", community_num)
    mod=community.modularity(comms, G, weight='weight')
    print("modularity=",mod)
    modularities.append(mod)
    unique_coms = np.unique(list(comms.values()))


    # Calculate the flow centrality of each node
    flow_centrality = nx.current_flow_closeness_centrality(Gmax, weight='weight')
    mean_flow_centrality = sum(flow_centrality.values())/len(flow_centrality)
    mean_flow_centralities.append(mean_flow_centrality)
    flow_centrality_values = list(flow_centrality.values())
    flow_centrality_list.append(flow_centrality_values)
    std_flow_centrality = np.std(flow_centrality_values)
    std_flow_centralities.append(std_flow_centrality)
    print("flow centrality=", mean_flow_centrality)
    print("std flow centrality=", std_flow_centrality)
    # Calculate the spectral gap of the network
    spectral_gap = nx.algebraic_connectivity(G, weight='weight')
    print("spectral gap=", spectral_gap)
    # Calculate the harmonic centrality of each node
    harmonic_centrality = nx.harmonic_centrality(G)
    mean_harmonic_centrality = sum(harmonic_centrality.values())/len(harmonic_centrality)
    mean_harmonic_centralities.append(mean_harmonic_centrality)
    harmonic_centrality_values = list(harmonic_centrality.values())
    harmonic_centrality_list.append(harmonic_centrality_values)
    std_harmonic_centrality = np.std(harmonic_centrality_values)
    std_harmonic_centralities.append(std_harmonic_centrality)
    print("harmonic centrality=", mean_harmonic_centrality)
    print("std harmonic centrality=", std_harmonic_centrality)
    #degree=nx.degree(G)
#     degree = {node: d for node, d in degree}

    cham_flow_cen=[]
    cham_bet_cen=[]
    for cham in ALL_CHAM_NUMS:
        try:
            ch_flo=flow_centrality[cham]
        except KeyError:
            ch_flo=np.nan
        cham_flow_cen.append(ch_flo)
        try:
            ch_bc=bet_cen[cham]
        except KeyError:
            ch_bc=np.nan
        cham_bet_cen.append(ch_bc)

    mean_chamber_flow_centrality=np.nanmean(cham_flow_cen)
    std_chamber_flow_centrality=np.nanstd(cham_flow_cen)
    mean_chamber_bet_centrality=np.mean(cham_bet_cen)
    std_chamber_bet_centrality=np.std(cham_bet_cen)

    cham_flow_centrality_list.append(cham_flow_cen)
    cham_mean_flow_centralities.append(mean_chamber_flow_centrality)
    cham_std_flow_centralities.append(std_chamber_flow_centrality)

    cham_betweenness_centrality_list.append(cham_bet_cen)
    cham_mean_betweenness_centrality.append(mean_chamber_bet_centrality)
    cham_std_betweenness_centrality.append(std_chamber_bet_centrality)

    junc_flow_cen=[]
    junc_bet_cen=[]

    for junc in ALL_JUNC_NUMS:
        try:    
            ju_flo=flow_centrality[junc]
        except KeyError:
            ju_flow=np.nan
        junc_flow_cen.append(ju_flo)
        try:
            ju_bc=bet_cen[junc]
        except KeyError:
            ju_bc=np.nan
        junc_bet_cen.append(ju_bc)

    mean_junc_flow_centrality=np.nanmean(junc_flow_cen)
    std_junc_flow_centrality=np.nanstd(junc_flow_cen)
    mean_junc_bet_centrality=np.nanmean(junc_bet_cen)
    std_junc_bet_centrality=np.nanstd(junc_bet_cen)

    junc_flow_centrality_list.append(junc_flow_cen)
    junc_mean_flow_centralities.append(mean_junc_flow_centrality)
    junc_std_flow_centralities.append(std_junc_flow_centrality)

    junc_betweenness_centrality_list.append(junc_bet_cen)
    junc_mean_betweenness_centrality.append(mean_junc_bet_centrality)
    junc_std_betweenness_centrality.append(std_junc_bet_centrality)

    ne_bet_cen=[]
    ne_flow_cen=[]
    for ne in ALL_NE_NUMS:
        try:
            try:
                ne_flo=flow_centrality[ne]
            except KeyError:
                ne_flo=np.nan
            ne_flow_cen.append(ne_flo)
            try:
                ne_bc=bet_cen[ne]
            except KeyError:
                ne_bc=np.nan
            ne_bet_cen.append(ne_bc)
        except KeyError:
            continue

    mean_ne_flow_centrality=np.nanmean(ne_flow_cen)
    std_ne_flow_centrality=np.nanstd(ne_flow_cen)

    mean_ne_bet_centrality=np.mean(ne_bet_cen)
    std_ne_bet_centrality=np.std(ne_bet_cen)

    ne_betweenness_centrality_list.append(ne_bet_cen)
    ne_mean_betweenness_centrality.append(mean_ne_bet_centrality)
    ne_std_betweenness_centrality.append(std_ne_bet_centrality)

    ne_flow_centrality_list.append(ne_flow_cen)
    ne_mean_flow_centrality.append(mean_ne_flow_centrality)
    ne_std_flow_centrality.append(std_ne_flow_centrality)
    #repeat for degree
    # Calculate the effective resistance distance between two nodes
#effective_resistance_distance = nx.effective_resistance(G, 1, 5)

analysis_df["betweenness_centrality_list"] = betweenness_centrality_list
analysis_df["mean_betweenness_centrality"] = mean_betweenness_centrality
analysis_df["std_betweenness_centrality"] = std_betweenness_centrality

analysis_df["degree_assortativity"] = degree_assortativity
analysis_df["global_assortativity"] = global_assortativity
analysis_df["cham_assortativity"] = cham_assortativity
analysis_df["junc_assortativity"] = junc_assortativity
analysis_df["ne_assortativity"] = ne_assortativity
analysis_df["end_assortativity"] = end_assortativity

analysis_df["efficiency"] = efficiency

analysis_df["communities"] = communities
analysis_df["modularities"] = modularities

analysis_df["flow_centrality_list"] = flow_centrality_list
analysis_df["mean_flow_centralities"] = mean_flow_centralities
analysis_df["std_flow_centralities"] = std_flow_centralities

analysis_df["spectral_gap"] = spectral_gap

analysis_df["harmonic_centrality_list"] = harmonic_centrality_list
analysis_df["mean_harmonic_centralities"] = mean_harmonic_centralities
analysis_df["std_harmonic_centralities"] = std_harmonic_centralities

analysis_df["cham_flow_centrality_list"] = cham_flow_centrality_list
analysis_df["cham_mean_flow_centralities"] = cham_mean_flow_centralities
analysis_df["cham_std_flow_centralities"] = cham_std_flow_centralities

analysis_df['cham_betweenness_centrality'] = cham_betweenness_centrality_list
analysis_df['cham_mean_betweenness_centrality'] = cham_mean_betweenness_centrality
analysis_df['cham_std_betweenness_centrality'] = cham_std_betweenness_centrality

analysis_df['junc_flow_centrality'] = junc_flow_centrality_list
analysis_df['junc_mean_flow_centralities'] = junc_mean_flow_centralities
analysis_df['junc_std_flow_centralities'] = junc_std_flow_centralities

analysis_df['junc_betweenness_centrality'] = junc_betweenness_centrality_list
analysis_df['junc_mean_betweenness_centrality'] = junc_mean_betweenness_centrality
analysis_df['junc_std_betweenness_centrality'] = junc_std_betweenness_centrality

analysis_df['ne_flow_centrality'] = ne_flow_centrality_list
analysis_df['ne_mean_flow_centrality'] = ne_mean_flow_centrality
analysis_df['ne_std_flow_centrality'] = ne_std_flow_centrality

analysis_df['ne_betweenness_centrality'] = ne_betweenness_centrality_list
analysis_df['ne_mean_betweenness_centrality'] = ne_mean_betweenness_centrality
analysis_df['ne_std_betweenness_centrality'] = ne_std_betweenness_centrality

analysis_df['junction_degree_assortativity']=junction_degree_assortativity
analysis_df['chamber_degree_assortativity']=chamber_degree_assortativity

analysis_df['JC_assortativity']=JC_assortativity
analysis_df['JN_assortativity']=JN_assortativity
analysis_df['JE_assortativity']=JE_assortativity

mean degree centrality= 0.13333333333333333
std dev degree centrality= 0.07766431633476231
clustering coefficient= 0.0
std dev clustering= 0.0
transitivity 0
square clustering 0.0
std dev square clustering 0.0
triangles= 0.0
closeness_centrality_spatial 0.03668459731572505
mean degree centrality= 0.05507246376811594
std dev degree centrality= 0.025268979382844482
clustering coefficient= 0.03985507246376811
std dev clustering= 0.10532204241846835
transitivity 0.05263157894736842
square clustering 0.007905138339920948
std dev square clustering 0.02561557588303502
triangles= 2.0
closeness_centrality_spatial 0.004279885750079498
mean degree centrality= 0.06463414634146343
std dev degree centrality= 0.025288343113814296
clustering coefficient= 0.04065040650406504
std dev clustering= 0.10267748731963486
transitivity 0.05714285714285714
square clustering 0.02103206090480746
std dev square clustering 0.03407676428483178
triangles= 2.0
closeness_centrality_spatial 0.0051257370499646375
mean deg

/tmp/ipykernel_1093446/112854948.py:282: RuntimeWarning: Mean of empty slice
  mean_ne_deg_centrality=np.nanmean(ne_degree_centrality)
/home/ll16598/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_1093446/112854948.py:284: RuntimeWarning: Mean of empty slice
  mean_ne_degree=np.nanmean(ne_degree)
/tmp/ipykernel_1093446/112854948.py:286: RuntimeWarning: Mean of empty slice
  mean_ne_closeness_centrality=np.nanmean(ne_closeness)
/tmp/ipykernel_1093446/112854948.py:288: RuntimeWarning: Mean of empty slice
  mean_ne_closeness_centrality_spatial=np.nanmean(ne_closeness_spatial)


closeness_centrality_spatial 0.00928767869015524
mean degree centrality= 0.019250253292806486
std dev degree centrality= 0.0073645074053597
clustering coefficient= 0.03309692671394799
std dev clustering= 0.09768772345812948
transitivity 0.037783375314861464
square clustering 0.03408028194639862
std dev square clustering 0.05030048243185999
triangles= 5.0
closeness_centrality_spatial 0.0036758106044782707
mean degree centrality= 0.01909854851031322
std dev degree centrality= 0.005795052776062569
clustering coefficient= 0.07597402597402599
std dev clustering= 0.13725282141977133
transitivity 0.08519269776876268
square clustering 0.03321761592317926
std dev square clustering 0.045717873696959475
triangles= 14.0
closeness_centrality_spatial 0.005666643146091467
mean degree centrality= 0.06386554621848739
std dev degree centrality= 0.04582722085524699
clustering coefficient= 0.057142857142857134
std dev clustering= 0.12562767579307546
transitivity 0.06896551724137931
square clustering 0.005

/home/ll16598/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ll16598/.local/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/ll16598/.local/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/ll16598/.local/lib/python3.8/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/ll16598/.local/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_1093446/112854948.py:229: RuntimeWarning: Mean of empty slice
  mean_cham_cluste

closeness_centrality_spatial 0.016455078617354374
mean degree centrality= 0.01786600496277915
std dev degree centrality= 0.007000861528370167
clustering coefficient= 0.08247863247863249
std dev clustering= 0.17628036777232894
transitivity 0.08860759493670886
square clustering 0.025896011733125633
std dev square clustering 0.04736200999609763
triangles= 14.0
closeness_centrality_spatial 0.008523841674264388
mean degree centrality= 0.017731557683674997
std dev degree centrality= 0.006135425507577708
clustering coefficient= 0.08841463414634149
std dev clustering= 0.14396601301131084
transitivity 0.1018867924528302
square clustering 0.03285215844633992
std dev square clustering 0.06088463348305685
triangles= 18.0
closeness_centrality_spatial 0.0076722542982328185
mean degree centrality= 0.015804935370152758
std dev degree centrality= 0.005076904322999558
clustering coefficient= 0.06900900900900901
std dev clustering= 0.1320680161121485
transitivity 0.08080808080808081
square clustering 0.0

...

spectral gap= 0.002117233636234133
harmonic centrality= 13.551578643578639
std harmonic centrality= 2.105620992569491
mean betweenness centrality= 0.07022144522144523
std dev betweenness centrality= 0.07278741343792376
['67', '70', '74', '104', '100', '103', '80', '35', '43', '26', '40', '36', '12', '95', '53', '45', '106', '7', '88', '86', '102', '93', '83', '69', '49', '44', '72', '107', '82', '22', '1', '51', '77', '97', '50', '78', '90', '38', '46', '48', '61', '33', '52', '98', '79', '8', '25', '73']
community_num= 10
modularity= 0.7596181948187802
flow centrality= 9.506822125298184e-05
std flow centrality= 1.8706276642185573e-05
spectral gap= 0.0001788350837163186
harmonic centrality= 16.011614900251256
std harmonic centrality= 2.6001310781535896
mean betweenness centrality= 0.063034012401101
std dev betweenness centrality= 0.048520871128439266
['32', '22', '86', '135', '35', '40', '36', '72', '33', '53', '81', '127', '139', '128', '97', '69', '87', '96', '98', '120', '79', '110'

community_num= 9
modularity= 0.6007729778301988
flow centrality= 0.00011322728483564133
std flow centrality= 2.6721107884178445e-05
spectral gap= 0.00023505657460689826
harmonic centrality= 20.531830030144633
std harmonic centrality= 3.1703401082313376
mean betweenness centrality= 0.17991056071551426
std dev betweenness centrality= 0.22765341790411187
['21', '8', '3', '6', '9', '2', '4', '5', '13']
community_num= 4
modularity= 0.6426755500299696
flow centrality= 0.0014372869085347323
std flow centrality= 0.0003476451601686756
spectral gap= 0.005387514344662209
harmonic centrality= 6.387134502923977
std harmonic centrality= 1.220922655855844
mean betweenness centrality= 0.11062801932367153
std dev betweenness centrality= 0.15315896796687248
['9', '42', '12', '2', '44', '24', '11', '4', '3', '39', '43', '23', '26', '47', '45', '52', '30', '6', '8', '37', '40', '18', '53', '48', '38', '28', '57', '13']
community_num= 8
modularity= 0.6582820538363715
flow centrality= 0.0004288817512751751


community_num= 8
modularity= 0.7765711664331825
flow centrality= 0.0002356942764709466
std flow centrality= 5.548155426217108e-05
spectral gap= 0.002051999251071896
harmonic centrality= 13.118438505938505
std harmonic centrality= 2.0824880349151695
mean betweenness centrality= 0.175
std dev betweenness centrality= 0.2775137634224452
['9', '5', '10']
community_num= 3
modularity= 0.45739192593838873
flow centrality= 0.006985853495621948
std flow centrality= 0.0016517726784079322
spectral gap= 0.024383819244459988
harmonic centrality= 4.6
std harmonic centrality= 0.9886859966642593
mean betweenness centrality= 0.16493506493506496
std dev betweenness centrality= 0.2137393007181919
['11', '8', '5', '25', '15', '17', '27', '22', '4', '10']
community_num= 6
modularity= 0.6873388065742188
flow centrality= 0.0003230513763930509
std flow centrality= 6.965839599773722e-05
spectral gap= 0.001037271935570605
harmonic centrality= 6.974134199134199
std harmonic centrality= 1.3593775199951925
mean bet

mean betweenness centrality= 0.12207792207792209
std dev betweenness centrality= 0.09535485758615624
['36', '18', '33', '8', '1', '32', '31', '37', '35', '16', '27', '25', '34', '40', '28']
community_num= 4
modularity= 0.41549025040618937
flow centrality= 0.0006440251065917958
std flow centrality= 0.00016749605341545852
spectral gap= 0.0027137412870070123
harmonic centrality= 8.063311688311687
std harmonic centrality= 1.324477569201816
mean betweenness centrality= 0.08721605488480133
std dev betweenness centrality= 0.06583576496407417
['50', '52', '51', '69', '8', '40', '63', '61', '59', '67', '16', '35', '54', '37', '68', '28', '58', '31', '36', '39', '47', '65', '45', '25', '66', '62', '18', '64', '1', '46', '33', '55']
community_num= 7
modularity= 0.7118391856183999
flow centrality= 0.0004701146875805432
std flow centrality= 0.0001326134965304281
spectral gap= 0.0035214944739010152
harmonic centrality= 12.107419712070872
std harmonic centrality= 1.7245648322429867
mean betweenness c

/tmp/ipykernel_1093446/112854948.py:690: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  analysis_df["communities"] = communities
/tmp/ipykernel_1093446/112854948.py:691: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  analysis_df["modularities"] = modularities
/tmp/ipykernel_1093446/112854948.py:693: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragme

In [37]:
JC_assortativity

[0.05855855855855855,
 0.02184087363494555,
 -0.031146767343086818,
 0.054187192118226604,
 0.00144927536231913,
 0.04454342984409738,
 0.027655724993538033,
 -0.05972797161442933,
 0.011515508111288052,
 0.03777242554191361,
 0.0,
 0.21939855993223212,
 0.1355240549828182,
 0.027624309392265116,
 0.03273619108551918,
 -7.098698733209342e-16,
 0.0,
 -0.01856148491879401,
 0.0727742888806793,
 0.03540138076220546,
 -0.023004059539918627,
 -0.03225806451612903,
 -0.047619047619048276,
 -0.052631578947368016,
 -0.0041284403669719924,
 -0.004402377283732602,
 -0.031496062992125706,
 -0.0019801980198021337,
 0.09620991253644273,
 0.05882352941176492,
 0.0,
 -0.040540540540541084,
 -0.06666666666666604,
 -0.04868913857677874,
 -0.05672969966629693,
 0.0,
 -7.569702440626075e-16,
 0.02191780821917832,
 0.06267806267806299,
 0.3120000000000003,
 0.14285714285714285,
 0.15151515151515146,
 0.07692307692307693,
 0.12436048799685165,
 0.08962868117797719,
 -0.15942028985507245,
 -0.15217391304347

In [38]:
#UNWEIGHTED RICH-CLUB ANALYSIS
rich_club=[]
for g in range(0,len(G_list)):
    G=G_list[g]
    new_g=G.copy(G)
    G=new_g
    G.remove_edges_from(nx.selfloop_edges(G))
    rc=nx.rich_club_coefficient(G, normalized=False, Q=100, seed=None)
    print('RICH_CLUB=', rc)
    rc1=list(rc.values())
    #print(rc1)
    rich_club.append(rc1)
analysis_df['rich_club']=rich_club

RICH_CLUB= {0: 0.13333333333333333, 1: 0.3333333333333333, 2: 0.3333333333333333}
RICH_CLUB= {0: 0.05507246376811594, 1: 0.09032258064516129, 2: 0.09032258064516129, 3: 0.16666666666666666}
RICH_CLUB= {0: 0.06463414634146342, 1: 0.09655172413793103, 2: 0.09655172413793103, 3: 0.1}
RICH_CLUB= {0: 0.08505747126436781, 1: 0.14210526315789473, 2: 0.14210526315789473, 3: 0.3333333333333333}
RICH_CLUB= {0: 0.05319148936170213, 1: 0.08199643493761141, 2: 0.08199643493761141, 3: 0.16666666666666666}
RICH_CLUB= {0: 0.0620782726045884, 1: 0.10666666666666667, 2: 0.10666666666666667, 3: 0.0}
RICH_CLUB= {0: 0.031092436974789917, 1: 0.047540983606557376, 2: 0.047540983606557376, 3: 0.05454545454545454, 4: 0.3333333333333333}
RICH_CLUB= {0: 0.02844774273345702, 1: 0.038299253489126904, 2: 0.038299253489126904, 3: 0.058823529411764705}
RICH_CLUB= {0: 0.019250253292806486, 1: 0.026843357118586476, 2: 0.026843357118586476, 3: 0.04411764705882353, 4: 0.0}
RICH_CLUB= {0: 0.019098548510313215, 1: 0.023239

/tmp/ipykernel_1093446/2497134369.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  analysis_df['rich_club']=rich_club


In [47]:
if WIDTH==True:
    analysis_df.to_csv(directory+"/"+'GRAPH_ANALYSIS_WIDTH.csv',index=False)#S+PARAMETERS
elif WIDTH==False:
    analysis_df.to_csv(directory+"/"+'GRAPH_ANALYSIS_BASE.csv',index=False)#S+PARAMETERS